# Train and export TensorFlow Serving model

In [1]:
def wrapper():

    import os
    import sys

    # This is a placeholder for a Google-internal import.

    import tensorflow as tf

    #import mnist_input_data
    from tensorflow.examples.tutorials.mnist import input_data as mnist_input_data

    from hops import serving

    training_iteration=1000
    model_version=1
    work_dir=os.getcwd()

    # Train model
    print('Training model...')
    mnist = mnist_input_data.read_data_sets(work_dir, one_hot=True)
    sess = tf.InteractiveSession()
    serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
    feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
    tf_example = tf.parse_example(serialized_tf_example, feature_configs)
    x = tf.identity(tf_example['x'], name='x')  # use tf.identity() to assign name
    y_ = tf.placeholder('float', shape=[None, 10])
    w = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    sess.run(tf.global_variables_initializer())
    y = tf.nn.softmax(tf.matmul(x, w) + b, name='y')
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    values, indices = tf.nn.top_k(y, 10)
    table = tf.contrib.lookup.index_to_string_table_from_tensor(
        tf.constant([str(i) for i in range(10)]))
    prediction_classes = table.lookup(tf.to_int64(indices))
    for _ in range(training_iteration):
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float')) 
    acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print('Training Accuracy: {}'.format(acc))
    print('Done training!')

    # Export model
    # WARNING(break-tutorial-inline-code): The following code snippet is
    # in-lined in tutorials, please update tutorial documents accordingly
    # whenever code changes.

    export_path = work_dir + '/model'
    print('Exporting trained model to: {}'.format(export_path))
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    # Build the signature_def_map.
    classification_inputs = tf.saved_model.utils.build_tensor_info(
        serialized_tf_example)
    classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
        prediction_classes)
    classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

    classification_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
              inputs={
                  tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                      classification_inputs
              },
              outputs={
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                      classification_outputs_classes,
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                      classification_outputs_scores
              },
              method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

    tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
    tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

    prediction_signature = (
          tf.saved_model.signature_def_utils.build_signature_def(
              inputs={'images': tensor_info_x},
              outputs={'scores': tensor_info_y},
              method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    builder.add_meta_graph_and_variables(
          sess, [tf.saved_model.tag_constants.SERVING],
          signature_def_map={
              'predict_images':
                  prediction_signature,
              tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                  classification_signature,
          },
          legacy_init_op=legacy_init_op)

    builder.save()

    print('Done exporting!')

    serving.export(export_path, "mnist", 1, overwrite=True)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1560176614871_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from hops import experiment

experiment.launch(wrapper, name='mnist serving example')

Finished Experiment 

'hdfs://10.0.2.15:8020/Projects/demo_deep_learning_admin000/Experiments/application_1560176614871_0001/launcher/run.1'

# Create Model Serving of Exported Model

In [3]:
from hops import serving

In [4]:
# Create serving
model_path="/Models/mnist/"
SERVING_NAME="mnist"
SERVING_VERSION=1
response = serving.create_or_update(model_path, SERVING_NAME, serving_type="TENSORFLOW", 
                                 model_version=SERVING_VERSION)

Creating a serving for model mnist ...
Serving for model mnist successfully created

In [5]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

mnist

In [6]:
# Get serving status
serving.get_status(SERVING_NAME)

'Stopped'

# Start Model Serving Server

In [7]:
if serving.get_status(SERVING_NAME) == 'Stopped':
    serving.start(SERVING_NAME)

Starting serving with name: mnist...
Serving with name: mnist successfully started

In [9]:
import time
time.sleep(10) # Let the serving startup correctly

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [10]:
import numpy as np
TOPIC_NAME = serving.get_kafka_topic(SERVING_NAME)
NUM_FEATURES=784

In [11]:
for i in range(20):
    data = {
                "signature_name": 'predict_images',
                "instances": [np.random.rand(784).tolist()]
            }
    response = serving.make_inference_request(SERVING_NAME, data)
    print(response)

{'predictions': [[0.0014034, 1.374e-08, 0.200702, 0.278433, 2.62939e-06, 0.515982, 0.000110427, 0.000536466, 0.00279624, 3.41006e-05]]}
{'predictions': [[0.00105801, 1.44569e-08, 0.370394, 0.293638, 1.62416e-06, 0.32803, 6.99966e-05, 0.00376709, 0.00293636, 0.000104489]]}
{'predictions': [[0.000664825, 4.21654e-08, 0.933665, 0.0423333, 1.82413e-07, 0.020572, 1.23976e-05, 6.47347e-05, 0.0026592, 2.8205e-05]]}
{'predictions': [[0.000324636, 2.48305e-08, 0.707619, 0.117427, 7.54794e-06, 0.169898, 2.80385e-05, 8.61907e-05, 0.00433046, 0.000278655]]}
{'predictions': [[0.00161419, 1.70108e-08, 0.278404, 0.166106, 3.7164e-06, 0.546742, 0.000127588, 0.000609636, 0.00624632, 0.000147593]]}
{'predictions': [[1.95921e-05, 1.39617e-09, 0.938718, 0.0441663, 5.77402e-08, 0.0165412, 1.19215e-05, 3.79244e-06, 0.000537965, 1.43126e-06]]}
{'predictions': [[0.00211452, 1.34315e-07, 0.669427, 0.245902, 7.54182e-07, 0.0784495, 0.000155536, 0.00103769, 0.00264703, 0.000265703]]}
{'predictions': [[0.00575027